In [3]:
from math import exp, log, sqrt, factorial
from scipy.stats import norm
import numpy as np

In [10]:
def repl_portfolio(fu, fd, u, d, S0, r, delta):
    Delta = (fu-fd) / (S0(u-d))
    Psi = (d*fu - u*fd) /(exp(r*delta) * (u-d))
    return Delta, Psi

def no_arb_val(Delta, Psi, S0):
    return Delta * S0 - Psi

def risk_neutral_prob(r, delta, u, d):
    return (exp(r*delta)-d) / (u-d)

def risk_neutral_value(p_star, fu, fd, r, delta):
    return exp(-r*delta) * (p_star * fu + (1-p_star) * fd)

def put_payoff(K, S):
    return max(K-S, 0)

def call_payoff(K, S):
    return max(S-K, 0)

def delta_hedge(fu, fd, u, d, S0):
    return (fu-fd) / (S0 * (u-d))

def lookback_call_payoff(K, S_max):
    return max(0, S_max - K)

def crr_binom(sigma, delta):
    return exp(sigma * sqrt(delta)), exp(-sigma * sqrt(delta))

def comb(N, k):
    return factorial(N) / (factorial(N-k) * factorial(k))

def risk_neutral_pricing(p, fN, r, u, d, delta):
    s = sum([comb(N, j) * (p**j) * (1-p)**(N-j) * fN[j] for j in range(0, N+1)])
    return exp(-r * N*delta) * s

def N_cdf(x):
    return norm(0, 1).cdf(x)

def black_sholes(N, delta, S0, K, r, q, sigma, opt_type="put"):
    T = N * delta
    d1 = (log(S0 / K) + (r - q + 0.5 * (sigma**2)) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    print(d1, d2)
    if opt_type == "put":
        return -S0 * exp(-q * T) * N_cdf(-d1) + K * exp(-r * T) * N_cdf(-d2)
        
    elif opt_type == "call":
        return S0 * exp(-q * T) * N_cdf(d1) - K * exp(-r * T) * N_cdf(d2)
        
    else:
        print("invalid type")
        return None
    
    

In [11]:
# 1

S0 = 50
u = 1.2
d = 0.8
r = 0.01
K = 50

# Risk neutral prob
p_star = risk_neutral_prob(r, 1/2, u, d)
print("P_star:", p_star)

# Payoff
fuu = put_payoff(K, (u**2) * S0)
fdd = put_payoff(K, (d **2) * S0)
fud = put_payoff(K, (u*d) * S0)

fu = risk_neutral_value(p_star, fuu, fud, r, 1/2)
fd = risk_neutral_value(p_star, fud, fdd, r, 1/2)
f0 = risk_neutral_value(p_star, fu, fd, r, 1/2)
print(f"""
fuu: {fuu}
fdd: {fdd}
fud: {fud}
fu: {fu}
fd: {fd}
""")

# Value of option
print("Value:", f0)

# initial delta hedge
Delta0 = delta_hedge(fu, fd, u, d, S0)
print("Delta0:", Delta0 * 100)

# Adjust hedge
Delta_tu = delta_hedge(fuu, fud, u, d, S0)
Delta_td = delta_hedge(fud, fdd, u, d, d*S0)
print("Delta_td", Delta_td * 100)

P_star: 0.5125313021485024

fuu: 0
fdd: 17.999999999999993
fud: 2.0
fu: 0.9700748751560944
fd: 9.750623959634117

Value: 5.224131634995086
Delta0: -43.902745422390126
Delta_td -99.99999999999997


In [17]:
# 2

S0 = 100
delta = 3/12
u = 1.1
d = 0.9 
r = 0.01
K = 95

# Backward induction
p_star = risk_neutral_prob(0.01, 3/12, 1.1, 0.9)

fuu = call_payoff(K, max(S0, u*S0, (u**2)*S0))
fud = call_payoff(K, max(S0, u*S0, u*d*S0))
fdu = call_payoff(K, max(S0, d*S0, u*d*S0))
fdd = call_payoff(K, max(S0, d*S0, (d**2)*S0))

fu = risk_neutral_value(p_star, fuu, fud, r, delta)
fd = risk_neutral_value(p_star, fud, fdd, r, delta)

# Value
#f0 = risk_neutral_value(p_star, fu, fd, r, delta)
f0 = exp(-r * delta * 2)
f0 = f0 * (p_star**2*fuu + p_star*(1-p_star)*(fud+fdu) +(1-p_star)**2 * fdd)
# Delta hedge
Delta = delta_hedge(call_payoff(K, max(S0, u*S0)), call_payoff(K, max(S0, d*S0)), u, d, S0)

print(f"""
p_star: {p_star}
fuu: {fuu}
fdd: {fdd}
fud: {fud}
fdu: {fdu}
fu: {fu}
fd: {fd}
""")

print("Value:", f0, "Delta:", 100 * Delta)



p_star: 0.5125156380289756
fuu: 26.000000000000014
fdd: 5
fud: 15.000000000000014
fdu: 5
fu: 20.586142277287642
fd: 10.099875104101603

Value: 12.94964110559295 Delta: 50.00000000000006


In [14]:
# 3

# Stock index
S0 = 2500
u, d  = 1.15, 0.9
r = 0.01
q = 0.03
K = 2500
delta = 3/12
# Compute Value of 6 month at-the-money American Call option
p_star = risk_neutral_prob(r-q, delta, u, d)
fuu = call_payoff(K, (u**2) * S0)
fdd = call_payoff(K, (d**2) *S0)
fud = call_payoff(K, (u*d) * S0)

fu = max(call_payoff(K, u*S0), risk_neutral_value(p_star, fuu, fud, r, delta))
fd = max(call_payoff(K, d*S0), risk_neutral_value(p_star, fud, fdd, r, delta))
f0 = max(call_payoff(K, S0), risk_neutral_value(p_star, fu, fd, r, delta))

print(f"""
    p_star: {p_star}
    fuu: {fuu} 
    fdd: {fdd}
    fud: {fud}
    fu: {fu}, pu: {call_payoff(K, u*S0)}
    fd: {fd}, pd: {call_payoff(K, d*S0)}
    f0: {f0}, p0: {call_payoff(K, S0)}
""")

# When Exercise? 
print("Exercise at maturity")

# Value of equivalent European Call?
f0_euro = exp(-2*r*delta) * (p_star**2*fuu + 2*p_star*(1-p_star)*fud + (1-p_star)**2*fdd)
print(f0_euro)
# How much pay for right to exercise early?
print(f0-f0_euro)


    p_star: 0.38004991677072936
    fuu: 806.2499999999995 
    fdd: 0
    fud: 87.5
    fu: 375.0, pu: 375.0
    fd: 33.17133563149852, pd: 0
    f0: 162.6760920577644, p0: 0

Exercise at maturity
156.89872643941945
5.777365618344959


In [53]:
# 4

S0 = 1.25
r = 0.01
q = 0.015
sigma = 0.15
delta = 2/12
K = 1.25
print(p_star)
N = 3
# 3-step CRR binomial model european at-the-money put
u, d = crr_binom(sigma, delta)
p_star = risk_neutral_prob(r-q, delta, u, d)
print(f"u: {u}, d: {d}")

fN = [ put_payoff(K, (u**j) * (d**(N-j) * S0)) for j in range(N+1) ]
print(f"fN: {fN}")
f0 = risk_neutral_pricing(p_star, fN, r, u, d, delta)
print(f"f0: {f0}")

# Black sholes
bs = black_sholes(N, delta, S0, K, r, q, sigma, opt_type="put")
print(f"Bs: {bs}")
print(f0 - bs)

0.47789841652564624
u: 1.0631511100344377, d: 0.9406000619870564
fN: [0.2097804250735864, 0.07424992251617968, 0, 0]
f0: 0.05857994708727634
0.029462782549439483 -0.07660323462854264
Bs: 0.05410598190001392
0.004473965187262417


In [65]:
# 5

r = np.array([[0.04, 0.02, 0.01]])
sigma = np.array([[0.2, 0.12, 0.09]])
corr = np.array([[1, 0.3, 0.1], [0.3, 1, 0.2], [0.1, 0.2, 1]])

cov = np.dot(sigma.T, sigma) * corr

w1 = np.array([[0.25, 0.25, 0.5]])
w2 = np.array([[1/6, 1/2, 1/3]])

r_hat1 = np.sum(np.inner(w1, r))
r_hat2 = np.sum(np.inner(w2, r))

var1 = np.sum(np.dot(w1.T, w1) * cov)
var2 = np.sum(np.dot(w2.T, w2) * cov)

std1 = np.sqrt(var1)
std2 = np.sqrt(var2)

print(r_hat1, r_hat2, var1, var2, std1, std2)

print("Standard Deviation for the first portfolio is smaller. It is therefore less risky and therefore the better choice")

#res = 0.0
#for i in range(3):
#    for j in range(3):
#        res += w1[0,i] * w1[0,j] * cov[i, j]
        
#print(res)

#res = 0.0
#for i in range(3):
#    for j in range(3):
#        res += w2[0,i] * w2[0,j] * cov[i, j]
        
#print(res)
        


0.02 0.02 0.007315 0.007731111111111111 0.08552777326693359 0.08792673717994494
Standard Deviation for the first portfolio is smaller. It is therefore less risky and therefore the better choice


In [57]:
var1

0.007315

In [58]:
var2

0.007731111111111111